# Split Movie Frames

The first step in training a cryo-CARE network is to split the direct detector acquisition movies. To that end we will first align the frames with [MotionCor2](https://www.nature.com/articles/nmeth.4193) and then split the frames into even and odd acquistions.

In [ ]:
from glob import glob
import mrcfile
import os
import sys
from os.path import join, basename, splitext
from utils import cd

import subprocess

import numpy as np
from tqdm import tqdm

from matplotlib import pyplot as plt

from utils import *

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# imports and settings for the GPU

os.environ["CUDA_VISIBLE_DEVICES"]="0" # <<< Set the GPU you want to use for the network here (0-7)

In [ ]:
# this assumes you have copied the cryoCARE scripts into the right location
data_path = 'frames/'

In [ ]:
# Read the list of mrc-files and display them.
files = glob(data_path + '*.tif') #change to mrc if mrc
files

In [ ]:
# Here we read out the pixel spacing from the imod-command.
# Pixel spacing (Angstroms) according to the header
p = !header -p {files[0]}

pixel_spacing = float(p[0].split()[0]) 

print('Pixel Spacing in Angstroms:', pixel_spacing)

## Align & Split

Each file gets aligned with MotionCor2 and the aligned stack is written to a `tmp` directory. Then the aligned stack is loaded and split in the two halves (even/odd) and written to the corresponding directories.

In [ ]:
#if all files are .tiff instead of .mrc --- convert them (this is not required by default!)

# for file in files:
#     #print(file)
#     cmd = 'tif2mrc ' + file + ' ' + file[:-3]+'mrc'
#     print(cmd)
#     subprocess.run(cmd, shell=True)      
# #set list of files back to only the MRC files
# # Read the list of mrc-files and display them.
# files = glob(data_path +'*.mrc') #change to mrc if mrc
# files

In [ ]:
# Create directories
if not os.path.isdir('tmp'):
    os.mkdir('tmp')
if not os.path.isdir(join(data_path, 'even')):
    os.mkdir(join(data_path, 'even'))
if not os.path.isdir(join(data_path, 'odd')):
    os.mkdir(join(data_path, 'odd'))
if not os.path.isdir(join(data_path, 'full')):
    os.mkdir(join(data_path, 'full'))

In [ ]:
def align_and_split(files):    
    # Align the frames with MotionCor2 and write out the aligned stack with '-OutStack 1'
    !MotionCor2 -InTiff frames/ -InSuffix .tif -OutMrc tmp/ -Patch 5 5 5 -OutStack 1 -Gpu 0 -Serial 1;  
    aligned_files = glob('tmp/*Stk.mrc') # get the algined files, that are still stack (not summed yet)
    for f in tqdm(aligned_files):
        print(f)
        aligned_stack = mrcfile.open(glob(f)[0], permissive=True)  
        # Save even frames
        save_mrc(join(data_path, 'even', splitext(basename(f))[0][:-4] + '.mrc'), np.sum(aligned_stack.data[::2], axis=0), pixel_spacing)
        # Save odd frames
        save_mrc(join(data_path, 'odd', splitext(basename(f))[0][:-4] + '.mrc'), np.sum(aligned_stack.data[1::2], axis=0), pixel_spacing)
        # Save odd frames
        save_mrc(join(data_path, 'full', splitext(basename(f))[0][:-4] + '.mrc'), np.sum(aligned_stack.data, axis=0), pixel_spacing)
    # Remove raw aligned files
    remove_files('tmp', extension='.mrc')       

In [ ]:
# MotionCor2 will create a log file 
align_and_split(files)

## Remove `tmp` Directory

In [ ]:
remove_files('tmp', extension='.log')
os.removedirs('tmp')